In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy as db
import psycopg2
import pandas as pd
import time
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Set your Spotify API credentials
client_id = '6c2ff081fe494968a2e3a618c678fd62'
client_secret = '99da4179a82f4f999058535d8d9b47dc'

# Initialize Spotify client
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, client_secret))

# To access tables in Database

In [3]:
# Database connection parameters
# Connect to DB in Elephant SQL database
db_params = {
    "host": "rain.db.elephantsql.com",
    "dbname": "auspovuc", 
    "user": "auspovuc",  
    "password": "bmJdG19Daw9rkEsJ3VnkefRGCBF_oy7F",  
}


# Establish connection to Elephant SQL database
try:
    conn = psycopg2.connect(**db_params)
    print("Connected to the database!")
    
    # Create a cursor
    cur = conn.cursor()
    
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)

Connected to the database!


# Creation of views for each condition and age group

This set of SQL query creates views for each combination of mental condition and age group. The views hold information pertaining the count of each music genre for each mental condition and the respective age group. The genres are ranked based on counts.

In [4]:
# Import time module
import time

# Start timer
start = time.process_time()

# Declare conditions
conditions = ['anxiety', 'depression', 'insomnia', 'ocd']

# Declare age_group
groups = ['early_years', 'young_adults', 'mature_adults', 'middle_age', 'elderly']

# Create Views for each condition and age_group, ranking genres
for condition in conditions:
    for age_group in groups:
        view_name = f"{age_group}_{condition}_genre"

        # Construct SQL query to create view
        query = f"""
            CREATE OR REPLACE VIEW {view_name} AS (
                SELECT genre, COUNT(*) AS count,
                    ROW_NUMBER() OVER(ORDER BY COUNT(*) DESC) AS row_n
                FROM (
                    SELECT UNNEST(ARRAY[freq1, freq2, freq3]) AS genre
                    FROM music_survey
                    WHERE {condition} >= 7
                    AND age_group = '{age_group}'
                ) AS genre_list
                WHERE genre IS NOT NULL
                GROUP BY genre
            )
        """
        
        # Execute the query to create the view
        cur.execute(query)
        conn.commit()

# Calculate and print elapsed time
print("Elapsed time:", time.process_time() - start)

Elapsed time: 0.0


In [5]:
# Ask user's age_group
age = input('Please indicate your age group:\n  1.0-17 \n  2.18-34 \n  3.35-59 \n  4.60-74 \n  5.76 and above\nEnter a number: ')

Please indicate your age group:
  1.0-17 
  2.18-34 
  3.35-59 
  4.60-74 
  5.76 and above
Enter a number: 2


In [6]:
# Ask if user has any of the 4 conditions
feels = input('How are you feeling today (select a number)?\n  1.Anxious \n  2.Depressed \n  3.Can\'t sleep \n  4.I have OCD \n  5.I feel great!\nEnter a number: ')

How are you feeling today (select a number)?
  1.Anxious 
  2.Depressed 
  3.Can't sleep 
  4.I have OCD 
  5.I feel great!
Enter a number: 1


# Retrieving recommended track based on inputs

This code segment is for the user input and query to be executed to retrieve the recommended track from the Database in Elephant SQL (recommended_tracks table-built through Spotify API). Based on the inputs it provides a music recommendation based on their age and condition.

In [7]:
# Define conditions
conditions = ['anxiety', 'depression', 'insomnia', 'ocd']

# Define age_group
groups = ['early_years', 'young_adults', 'mature_adults', 'middle_age', 'elderly']

# Ensure print dataframe with full column width for url display without truncation
pd.set_option('display.max_colwidth', None)  

# look for top choice of genre based on user input (*SQL INJECTION RISK)
if feels in ['1', '2', '3', '4']:
    if age in ['1', '2', '3', '4', '5']:
        x = int(feels)
        y = int(age)
        # query recommended_tracks for genre from specified view
        cur.execute(f"""
                SELECT t.track_name, t.artist, t.album, t.genre, t.preview_url 
                FROM recommended_tracks t
                JOIN {groups[y-1]}_{conditions[x-1]}_genre a
                ON a.genre = t.genre
                WHERE a.row_n = 1 ;
                """)
        conn.commit()
        result = cur.fetchall()
        #convert result to a dataframe and print    
        df = pd.DataFrame(result, columns=['Song', 'Artist', 'Album', 'Genre', 'Preview URL'])
        print('Have a listen to these songs. They just might make you feel better!\n\n', df)
    else:
        print('Hmm, you\'re good, you don\'t need help.')
    
elif feels == '5':
    # Define seed genres
    seed_genres = ['pagode']
    # Get recommended tracks for each seed genre
    for seed_genre in seed_genres:
        recommended_tracks = sp.recommendations(seed_genres=[seed_genre], limit=3, country="SG")
        for track in recommended_tracks["tracks"]:
            track_data = {
                        "track_name": track["name"],
                        "artist": ", ".join(artist["name"] for artist in track["artists"]),
                        "album": track["album"]["name"],
                        "genre": seed_genre,
                        "preview_url": track["preview_url"]
                            }
        # Create a DataFrame from the track data, suggest random track from top 3
        random_track = {random.randint(0,3)}
        df = pd.DataFrame(track_data, index=[random_track])
        print('Check this out!\n', df)
else:
    print('Hmm, you\'re good, you don\'t need help.')

Have a listen to these songs. They just might make you feel better!

                                     Song          Artist  \
0                         Tuesday's Gone  Lynyrd Skynyrd   
1  Don't Look Back in Anger - Remastered           Oasis   
2                    Steady, As She Goes  The Raconteurs   

                                            Album Genre  \
0               Pronounced' Leh-'Nerd 'Skin-'Nerd  rock   
1  (What's The Story) Morning Glory? [Remastered]  rock   
2                             Broken Boy Soldiers  rock   

                                                                                                   Preview URL  
0                                                                                                         None  
1  https://p.scdn.co/mp3-preview/8c355933aec856bfddb7fdeca6a47868e3e33da0?cid=6c2ff081fe494968a2e3a618c678fd62  
2  https://p.scdn.co/mp3-preview/cc73c19243cd3f3a33b726d2bceec7d49429abc7?cid=6c2ff081fe494968a2e3a618c678fd62  


In [8]:
# close connection to db
conn.close()